In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import re
from collections import Counter
import random

# --- 유틸리티 함수 ---
def mean_absolute_percentage_error(y_true, y_pred):
    """
    MAPE를 계산합니다. 0으로 나누는 오류를 방지합니다.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return 0.0
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

# --- 장치 설정 (GPU 사용 가능 시) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 데이터 전처리 및 어휘 구축 ---
def preprocess_text(text):
    """
    텍스트를 소문자로 변환하고, 알파벳, 숫자, 공백만 남긴 후 단어 단위로 분리합니다 (토큰화).
    """
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text.split()

class Vocabulary:
    """
    텍스트 데이터로부터 어휘를 구축하고, 단어를 정수 ID로 변환합니다.
    """
    def __init__(self, min_freq):
        self.stoi = {"<PAD>": 0, "<UNK>": 1} # string_to_int: 패딩 토큰과 알 수 없는 단어 토큰 정의
        self.itos = {0: "<PAD>", 1: "<UNK>"} # int_to_string
        self.freq = Counter()
        self.min_freq = min_freq
    
    def build_vocabulary(self, text_list):
        """
        주어진 텍스트 리스트를 기반으로 어휘를 구축합니다.
        min_freq보다 적게 나타나는 단어는 <UNK> 토큰으로 처리됩니다.
        """
        for text in text_list:
            self.freq.update(text)
        
        idx = 2 # <PAD>, <UNK> 다음 인덱스부터 시작
        for word, count in self.freq.items():
            if count >= self.min_freq:
                self.stoi[word] = idx
                self.itos[idx] = word
                idx += 1
    
    def numericalize(self, text):
        """
        텍스트(단어 리스트)를 정수 ID 시퀀스로 변환합니다.
        """
        return [self.stoi.get(token, self.stoi["<UNK>"]) for token in text]

# --- 파일 로드 ---
# 'review.json' 파일이 현재 스크립트와 동일한 디렉토리에 있어야 합니다.
df = pd.read_json('review.json', lines=True)

# --- 필요한 컬럼 추출 및 인코딩 ---
df_processed = df[['user_id', 'business_id', 'stars', 'text']].copy()

user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

df_processed.loc[:, 'user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed.loc[:, 'business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

# --- 텍스트 전처리 및 어휘 구축 실행 ---
all_texts = df_processed['text'].apply(preprocess_text).tolist()
min_word_freq = 5 # 최소 단어 빈도수 설정 (조정 가능)
vocab = Vocabulary(min_word_freq)
vocab.build_vocabulary(all_texts)
vocab_size = len(vocab.stoi)
print(f"Vocabulary size: {vocab_size}")

# 리뷰 텍스트를 정수 ID 시퀀스로 변환하고, 패딩/트렁케이션 적용
MAX_REVIEW_LEN = 100 # 리뷰 텍스트의 최대 길이 (조정 가능)
df_processed.loc[:, 'numericalized_text'] = df_processed['text'].apply(vocab.numericalize)
df_processed['numericalized_text'] = df_processed['numericalized_text'].apply(
    lambda x: x[:MAX_REVIEW_LEN] if len(x) > MAX_REVIEW_LEN else x + [vocab.stoi["<PAD>"]] * (MAX_REVIEW_LEN - len(x))
)

# --- 데이터 분할 (7:1:2 비율) ---
# 먼저 전체에서 테스트 세트 (20%) 분리
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)

# 남은 train_val_df (80%)에서 학습 세트 (70%)와 검증 세트 (10%) 분리
# train_val_df는 전체의 80%이므로, 7:1 비율은 train_val_df의 7/8과 1/8이 됩니다.
train_df, val_df = train_test_split(train_val_df, test_size=1/8, random_state=42) 

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

# --- PyTorch Dataset 및 DataLoader 정의 ---
class AFRAMDataset(Dataset):
    """
    AFRAM 모델 학습을 위한 PyTorch Dataset 클래스.
    사용자 ID, 사업체 ID, 수치화된 리뷰 텍스트, 평점을 반환합니다.
    """
    def __init__(self, df):
        self.user_ids = torch.tensor(df['user_encoded'].values, dtype=torch.long)
        self.business_ids = torch.tensor(df['business_encoded'].values, dtype=torch.long)
        self.reviews = torch.tensor(np.array(df['numericalized_text'].tolist()), dtype=torch.long)
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)

    def __len__(self):
        return len(self.stars)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.business_ids[idx], self.reviews[idx], self.stars[idx]

# --- Dataset 객체 생성 (DataLoader보다 먼저 정의되어야 함!) ---
train_dataset = AFRAMDataset(train_df)
val_dataset = AFRAMDataset(val_df)
test_dataset = AFRAMDataset(test_df)

# --- AFRAM 모델 아키텍처 정의 ---
class TextEncoderWithAttention(nn.Module):
    """
    리뷰 텍스트에서 CNN, LSTM, 어텐션 메커니즘을 사용하여 특징을 추출합니다.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim, dropout_rate):
        super(TextEncoderWithAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # Convolutional Layer (CNN)
        self.conv = nn.Conv1d(in_channels=embedding_dim, out_channels=hidden_dim, kernel_size=3, padding=1)
        # Bidirectional LSTM Layer
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, bidirectional=True)
        
        # Attention Layer (Bahdanau-style Additive Attention)
        self.attn_proj = nn.Linear(hidden_dim * 2, hidden_dim * 2) # Bi-LSTM의 출력 차원에 맞춤
        self.v = nn.Parameter(torch.rand(hidden_dim * 2, 1)) # 어텐션 가중치 벡터 (학습 가능)
        
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, text_seq):
        # text_seq: (batch_size, seq_len)
        embedded = self.embedding(text_seq) # (batch_size, seq_len, embedding_dim)
        embedded = embedded.permute(0, 2, 1) # Conv1d를 위해 차원 변경 (batch_size, embedding_dim, seq_len)
        
        conv_out = torch.relu(self.conv(embedded)) # (batch_size, hidden_dim, seq_len)
        conv_out = conv_out.permute(0, 2, 1) # LSTM을 위해 차원 변경 (batch_size, seq_len, hidden_dim)

        lstm_out, _ = self.lstm(self.dropout(conv_out)) # (batch_size, seq_len, hidden_dim * 2) (Bi-LSTM)
        
        # 어텐션 메커니즘 적용
        attn_weights = torch.tanh(self.attn_proj(lstm_out)) # (batch_size, seq_len, hidden_dim * 2)
        v_expanded = self.v.unsqueeze(0).expand(attn_weights.shape[0], -1, -1) # 배치 크기에 맞게 v 확장
        
        scores = torch.bmm(attn_weights, v_expanded) # 어텐션 스코어 계산 (batch_size, seq_len, 1)
        attention_weights = torch.softmax(scores, dim=1) # 소프트맥스로 가중치 정규화 (sum=1)
        
        # 문맥 벡터 (가중합) 계산: 어텐션 가중치를 적용한 LSTM 출력의 가중 평균
        context_vector = torch.sum(lstm_out * attention_weights, dim=1) # (batch_size, hidden_dim * 2)
        
        return context_vector # 이 벡터가 리뷰의 "측면 특징"을 나타냅니다.

class AFRAMModel(nn.Module):
    """
    AFRAM 논문의 전체 모델 구조를 구현합니다.
    사용자-사업체 상호작용과 리뷰 텍스트 특징을 결합하여 평점을 예측합니다.
    """
    def __init__(self, num_users, num_businesses, vocab_size, embedding_dim,
                 text_encoder_hidden_dim, user_item_mlp_dims, final_mlp_dims, dropout_rate):
        super(AFRAMModel, self).__init__()
        
        # 사용자 및 사업체 임베딩 레이어
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        
        # 리뷰 텍스트를 인코딩하는 모듈 (어텐션 포함)
        self.review_encoder = TextEncoderWithAttention(vocab_size, embedding_dim, text_encoder_hidden_dim, dropout_rate)

        # 사용자-사업체 상호작용 MLP (논문의 Customer-Restaurant Interaction Module)
        user_item_mlp_input_dim = embedding_dim * 2
        user_item_layers = []
        for dim in user_item_mlp_dims:
            user_item_layers.append(nn.Linear(user_item_mlp_input_dim, dim))
            user_item_layers.append(nn.ReLU())
            user_item_mlp_input_dim = dim
        self.user_item_mlp = nn.Sequential(*user_item_layers)
        self.user_item_mlp_output_dim = user_item_mlp_dims[-1] if user_item_mlp_dims else embedding_dim * 2

        # 최종 평점 예측 MLP (논문의 Rating Prediction Module)
        final_mlp_input_dim = self.user_item_mlp_output_dim + \
                              text_encoder_hidden_dim * 2 # review_encoder의 출력 차원 (Bi-LSTM이므로 hidden_dim * 2)
        
        final_layers = []
        for dim in final_mlp_dims:
            final_layers.append(nn.Linear(final_mlp_input_dim, dim))
            final_layers.append(nn.ReLU())
            final_mlp_input_dim = dim
        final_layers.append(nn.Linear(final_mlp_input_dim, 1)) # 최종 출력은 평점 (1차원)
        self.prediction_mlp = nn.Sequential(*final_layers)

    def forward(self, user_ids, business_ids, reviews):
        # 사용자 및 사업체 임베딩 벡터 가져오기
        user_vec = self.user_embedding(user_ids)
        business_vec = self.business_embedding(business_ids)
        
        # 사용자-사업체 임베딩을 결합하고 MLP에 통과시켜 상호작용 특징 생성
        user_item_combined = torch.cat((user_vec, business_vec), dim=1)
        user_item_features = self.user_item_mlp(user_item_combined)

        # 리뷰 텍스트를 리뷰 인코더에 통과시켜 텍스트 특징(측면 특징) 추출
        review_features = self.review_encoder(reviews)
        
        # 상호작용 특징과 리뷰 텍스트 특징을 결합
        combined_features = torch.cat((user_item_features, review_features), dim=1)
        
        # 최종 평점 예측 MLP에 통과시켜 결과 반환
        predicted_rating = self.prediction_mlp(combined_features)
        return predicted_rating.squeeze() # 1차원 평점 반환을 위해 차원 축소

# --- 모델 학습 및 평가 (단일 파라미터 세트) ---
# 최적 또는 기본적으로 사용할 하이퍼파라미터 설정
# 이 값들은 논문이나 일반적인 딥러닝 모델에서 좋은 성능을 보이는 값들입니다.
# 필요에 따라 이 값을 직접 변경하여 실험할 수 있습니다.
params = {
    'embedding_dim': 64,
    'text_encoder_hidden_dim': 128,
    'learning_rate': 0.001,
    'batch_size': 256,
    'user_item_mlp_dims': [128, 64],
    'final_mlp_dims': [64, 32],
    'dropout_rate': 0.2
}

print(f"\n--- Starting training with fixed parameters ---")
print(f"Parameters: {params}")

# 파라미터 언팩
embedding_dim = params['embedding_dim']
text_encoder_hidden_dim = params['text_encoder_hidden_dim']
learning_rate = params['learning_rate']
batch_size = params['batch_size']
user_item_mlp_dims = params['user_item_mlp_dims']
final_mlp_dims = params['final_mlp_dims']
dropout_rate = params['dropout_rate']

epochs = 50 # 최대 에폭 수
patience = 10 # 조기 종료를 위한 검증 성능 개선 대기 에폭 수
min_delta = 0.0005 # 성능 개선으로 인정할 최소 변화량

best_val_rmse = float('inf')
epochs_no_improve = 0
model_save_path = 'best_afram_model.pt' # 최적 모델 저장 경로

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 인스턴스 생성 및 GPU로 이동
model = AFRAMModel(num_users, num_businesses, vocab_size, embedding_dim,
                   text_encoder_hidden_dim, user_item_mlp_dims, final_mlp_dims, dropout_rate).to(device)

criterion = nn.MSELoss() # 손실 함수: MSE
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # 옵티마이저: Adam

# --- 학습 루프 (조기 종료 포함) ---
for epoch in range(epochs):
    model.train() # 모델을 학습 모드로 설정
    total_train_loss = 0
    for user_ids, business_ids, reviews, stars in train_loader:
        # 데이터를 GPU로 이동
        user_ids, business_ids, reviews, stars = user_ids.to(device), business_ids.to(device), reviews.to(device), stars.to(device)
        
        optimizer.zero_grad() # 옵티마이저의 기울기 초기화
        predictions = model(user_ids, business_ids, reviews) # 예측 수행
        loss = criterion(predictions, stars) # 손실 계산
        loss.backward() # 역전파
        optimizer.step() # 파라미터 업데이트
        total_train_loss += loss.item()

    model.eval() # 모델을 평가 모드로 설정
    total_val_loss = 0
    val_predictions = []
    val_true_ratings = []
    with torch.no_grad(): # 기울기 계산 비활성화 (메모리 절약, 속도 향상)
        for user_ids, business_ids, reviews, stars in val_loader:
            # 데이터를 GPU로 이동
            user_ids, business_ids, reviews, stars = user_ids.to(device), business_ids.to(device), reviews.to(device), stars.to(device)
            
            predictions = model(user_ids, business_ids, reviews)
            loss = criterion(predictions, stars)
            total_val_loss += loss.item()
            val_predictions.extend(predictions.tolist())
            val_true_ratings.extend(stars.tolist())

    current_val_rmse = np.sqrt(mean_squared_error(val_true_ratings, val_predictions))

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {total_train_loss / len(train_loader):.4f}, "
          f"Val Loss: {total_val_loss / len(val_loader):.4f}, Val RMSE: {current_val_rmse:.4f}")

    # 조기 종료 로직
    # 현재 검증 RMSE가 이전까지의 최고 검증 RMSE보다 min_delta 이상 개선되었다면
    if current_val_rmse < best_val_rmse - min_delta:
        best_val_rmse = current_val_rmse
        epochs_no_improve = 0 # 개선되었으니 대기 카운트 초기화
        torch.save(model.state_dict(), model_save_path) # 최적 모델 저장
        print(f"  --> Validation RMSE improved. Model saved: {best_val_rmse:.4f}")
    else:
        epochs_no_improve += 1 # 개선되지 않았으니 대기 카운트 증가
        if epochs_no_improve == patience: # 대기 카운트가 patience에 도달하면
            print(f"  Early stopping! No improvement in RMSE for {patience} epochs.")
            break # 학습 중단

# --- 최종 모델 테스트 ---
if os.path.exists(model_save_path):
    model.load_state_dict(torch.load(model_save_path))
    print(f"\nLoaded best model weights from {model_save_path}")
else:
    print(f"\nCould not find best model weights at '{model_save_path}'. Testing with current model state.")

model.eval() # 모델을 평가 모드로 설정
test_predictions = []
true_ratings = []

with torch.no_grad():
    for user_ids, business_ids, reviews, stars in test_loader:
        user_ids, business_ids, reviews, stars = user_ids.to(device), business_ids.to(device), reviews.to(device), stars.to(device)
        predictions = model(user_ids, business_ids, reviews)
        test_predictions.extend(predictions.tolist())
        true_ratings.extend(stars.tolist())

mse = mean_squared_error(true_ratings, test_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(true_ratings, test_predictions)
mape = mean_absolute_percentage_error(true_ratings, test_predictions)

print(f"\n--- Final Model Performance on Test Set ---")
print(f"Used Hyperparameters: {params}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Using device: cuda
Vocabulary size: 51962
전체 데이터 수: 447796
학습 데이터 수: 313456 (70.00%)
검증 데이터 수: 44780 (10.00%)
테스트 데이터 수: 89560 (20.00%)

--- Starting training with fixed parameters ---
Parameters: {'embedding_dim': 64, 'text_encoder_hidden_dim': 128, 'learning_rate': 0.001, 'batch_size': 256, 'user_item_mlp_dims': [128, 64], 'final_mlp_dims': [64, 32], 'dropout_rate': 0.2}
Epoch 1/50, Train Loss: 1.2615, Val Loss: 1.0294, Val RMSE: 1.0146
  --> Validation RMSE improved. Model saved: 1.0146
Epoch 2/50, Train Loss: 0.9329, Val Loss: 0.9273, Val RMSE: 0.9630
  --> Validation RMSE improved. Model saved: 0.9630
Epoch 3/50, Train Loss: 0.8216, Val Loss: 0.8727, Val RMSE: 0.9342
  --> Validation RMSE improved. Model saved: 0.9342
Epoch 4/50, Train Loss: 0.7385, Val Loss: 0.8407, Val RMSE: 0.9169
  --> Validation RMSE improved. Model saved: 0.9169
Epoch 5/50, Train Loss: 0.6735, Val Loss: 0.8232, Val RMSE: 0.9073
  --> Validation RMSE improved. Model saved: 0.9073
Epoch 6/50, Train Loss: 0.615